To run writeprints, we have to clone its repository and change the feature_extractor.py file. We should modify the imports:
```python
from tensorflow.keras.preprocessing import text as tx
```


In [1]:
!git clone https://github.com/shaoormunir/writeprints.git

Cloning into 'writeprints'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 163 (delta 75), reused 124 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (163/163), 786.47 KiB | 8.19 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
%cd writeprints
!pip install ./

/content/writeprints
Processing /content/writeprints
  Preparing metadata (setup.py) ... done
  Created wheel for writeprints: filename=writeprints-0.2.0-py3-none-any.whl size=74939 sha256=c0b498c9661b82e6f1976be1763a4dccf03746a2ab290c3607b29272dd4cb7c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-2jh7ym3z/wheels/a4/2c/73/6a36040c72244f54973320c29b0176055e91ee0549afca141a
Successfully built writeprints


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import pickle
import nltk
nltk.download('punkt')
from google.colab import userdata
import google.generativeai as genai
from writeprints.text_processor import Processor

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
%cd ..

/content


In [47]:
# read dataframes

train_df = pd.read_csv("train_blog.csv")
test_df = pd.read_csv("test_blog.csv")
val_df = pd.read_csv("validation_blog.csv")

In [48]:
train_df

,style,text,category
0,blog5546,"At any rate, there was no keeping 20-year-old ...",blog
1,blog5546,I have also been told that a portion of the Hi...,blog
2,blog5546,"Even so, Mantel was disappointed with the over...",blog
3,blog5546,"Lilly, who is childless, began writing poetry ...",blog
4,blog5546,"""People started moving up here in the late '50...",blog
...,...,...,...
5541,blog30407,Also I'm concerned that I'm not eating enough ...,blog
5542,blog30407,Somehow I became fascinated with this little G...,blog
5543,blog30407,"With exercise, once you do it, it's done. Poof...",blog
5544,blog30407,"I'm buckling them in the middle row, not the l...",blog


In [31]:
# rename columns cause processor expects text

'''train_df.rename(columns={"review": "text"}, inplace=True)
val_df.rename(columns={"review": "text"}, inplace=True)
test_df.rename(columns={"review": "text"}, inplace=True)
train_df'''

,Unnamed: 0,reviewer,movie,rating,author_id,text,sentiment,sentiment_id
0,10010,Hitchcoc,Betty Boop's Ker-Choo (1933),7,6,Betty is entered in an auto race. She is late ...,positive,1
1,10740,boblipton,Miss Tulip Stays the Night (1955),5,7,Patrick Holt and Diana Dors arrive at their ne...,negative,0
2,6562,SnoopyStyle,Thunderstruck (I) (2012),4,4,Brian Newall is a hopelessly uncoordinated hig...,negative,0
3,11054,boblipton,Salt Lake Diversions (1943),5,7,James A. Fitzpatrick sends Charles P. Boyle an...,negative,0
4,282,MartinHafer,Scooby's Laff-A Lympics (1977–1979),1,0,"Okay,...I'll admit it. I hold a grudge against...",negative,0
...,...,...,...,...,...,...,...,...
10795,7532,Horst_In_Translation,The Magnetic Telescope (1942),4,5,It is nothing really new that Superman occasio...,negative,0
10796,9168,Hitchcoc,"The Twilight Zone: The Mirror (1961) Season 3,...",5,6,There's no suspense. It's another one of those...,negative,0
10797,4260,bkoganbing,Miss Marple: They Do It with Mirrors (1991 TV ...,6,2,Joan Hickson as Agatha Christie's intrepid Ms....,positive,1
10798,12477,claudio_carvalho,Tie han rou qing (1974),3,8,"The powerful mobster Leung, who is protected b...",negative,0


In [49]:
processor = Processor(flatten = True)

wp_train_df =  processor.extract_df(train_df)
wp_test_df =  processor.extract_df(test_df)
wp_val_df =  processor.extract_df(val_df)
wp_train_df

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


  0%|          | 0/5546 [00:00<?, ?it/s]

  0%|          | 0/1389 [00:00<?, ?it/s]

  0%|          | 0/981 [00:00<?, ?it/s]

,characters_count,average_characters_per_word,letters_frequency:a,letters_frequency:b,letters_frequency:c,letters_frequency:d,letters_frequency:e,letters_frequency:f,letters_frequency:g,letters_frequency:h,...,function_words_frequencies:that'll,function_words_frequencies:that'd,function_words_frequencies:might've,function_words_frequencies:must've,function_words_frequencies:havent,function_words_frequencies:could've,function_words_frequencies:unique,function_words_frequencies:should've,function_words_frequencies:ours,function_words_frequencies:oneself
0,282,4.089286,0.090047,0.009479,0.014218,0.033175,0.142180,0.033175,0.009479,0.066351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,660,5.009091,0.096591,0.024621,0.024621,0.039773,0.106061,0.013258,0.011364,0.068182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,501,4.905882,0.062972,0.017632,0.017632,0.047859,0.118388,0.012594,0.020151,0.042821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,444,4.907895,0.096970,0.015152,0.012121,0.033333,0.084848,0.012121,0.015152,0.051515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,324,4.416667,0.082305,0.020576,0.020576,0.041152,0.148148,0.032922,0.020576,0.028807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,113,3.956522,0.098765,0.024691,0.024691,0.037037,0.086420,0.012346,0.061728,0.024691,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5542,331,4.824561,0.076923,0.011538,0.026923,0.011538,0.138462,0.019231,0.030769,0.042308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5543,70,4.461538,0.038462,0.019231,0.038462,0.038462,0.096154,0.019231,0.019231,0.038462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5544,144,4.000000,0.037037,0.037037,0.018519,0.037037,0.129630,0.009259,0.009259,0.055556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
wp_train_df['style'] = train_df["style"]
wp_test_df['style'] = test_df["style"]
wp_val_df['style'] = val_df["style"]

In [51]:
wp_train_df

,characters_count,average_characters_per_word,letters_frequency:a,letters_frequency:b,letters_frequency:c,letters_frequency:d,letters_frequency:e,letters_frequency:f,letters_frequency:g,letters_frequency:h,...,function_words_frequencies:that'd,function_words_frequencies:might've,function_words_frequencies:must've,function_words_frequencies:havent,function_words_frequencies:could've,function_words_frequencies:unique,function_words_frequencies:should've,function_words_frequencies:ours,function_words_frequencies:oneself,style
0,282,4.089286,0.090047,0.009479,0.014218,0.033175,0.142180,0.033175,0.009479,0.066351,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog5546
1,660,5.009091,0.096591,0.024621,0.024621,0.039773,0.106061,0.013258,0.011364,0.068182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog5546
2,501,4.905882,0.062972,0.017632,0.017632,0.047859,0.118388,0.012594,0.020151,0.042821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog5546
3,444,4.907895,0.096970,0.015152,0.012121,0.033333,0.084848,0.012121,0.015152,0.051515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog5546
4,324,4.416667,0.082305,0.020576,0.020576,0.041152,0.148148,0.032922,0.020576,0.028807,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog5546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5541,113,3.956522,0.098765,0.024691,0.024691,0.037037,0.086420,0.012346,0.061728,0.024691,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog30407
5542,331,4.824561,0.076923,0.011538,0.026923,0.011538,0.138462,0.019231,0.030769,0.042308,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog30407
5543,70,4.461538,0.038462,0.019231,0.038462,0.038462,0.096154,0.019231,0.019231,0.038462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog30407
5544,144,4.000000,0.037037,0.037037,0.018519,0.037037,0.129630,0.009259,0.009259,0.055556,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blog30407


In [54]:
len(wp_train_df.iloc[4]["text"])

324

In [55]:
wp_val_df.to_csv("wp_validation_blog.csv", index=False)
wp_test_df.to_csv("wp_test_blog.csv", index=False)
wp_train_df.to_csv("wp_train_blog.csv", index=False)